In [1]:
from Bio import SeqIO
from Bio import Seq
import pandas as pd
import numpy as np
import random
import csv
import re
import os

In [2]:
def seqToString(motif):
    """
    motif: a Seq that represents the motif 
    
    Returns the String representation of the motif
    """
    i = 0
    string = ""
    length = motif.__len__()
    while i < length:
        string += motif.__getitem__(i)
        i += 1
    return string

In [3]:
def getNegative(pos_seq):
    """
    pos_seq = dna sequence in the positive direction reading from the file
    
    Returns the negative counterpart of the positive sequence.
    """
    dict = {"A":'T','T':'A','G':'C','C':'G','-':'-','N':'N'}
    negative = ""
    last_index = len(pos_seq) - 1
    while last_index > -1:
        negative += dict[pos_seq[last_index].upper()]
        last_index -= 1
    return negative

In [4]:
def randomize_strand(length):
    """
    length: the number of randomly chosen strand directions
    
    Returns a numpy array of randomly chosen strand directions of specified length.
    """
    strands = []
    for i in np.arange(length):
        r = random.random()
        if r < 0.5:
            strands += ["positive"]
        else:
            strands += ["negative"]
    return strands
    

In [29]:
def randomize_species(species_count, species_array):
    """
    species_count: The number of randomized species we want in our outputted csv
    species_array: The array of all the different species in the non thresholded file
    
    Returns an array of length "species_count" of randomly selected species from species_array
    """
    random_numbs = []
    random_spec = []
    for i in np.arange(species_count):
        random_numbs += [random.randint(0, len(species_array) - 1)]
    for num in random_numbs:
        random_spec += [species_array[num]]     
    return random_spec

In [37]:
no_thresh = pd.read_csv("~/motif_extraction/data/sub_no_map_motif_bcd_no_threshold/VT6436.fa.csv") 
species_array = no_thresh.drop_duplicates("species")["species"].values
print(species_array)
randomized_species = randomize_species(10, species_array)
randomized_species
subsetted = no_thresh.groupby("species").max()
rand_spec = pd.DataFrame(randomized_species, columns = ["species"])
merged = pd.merge(rand_spec, subsetted, on = ['species'])
merged

['VT6436|1|MEMB002A|+|2701' 'VT6436|1|MEMB002B|-|2408'
 'VT6436|1|MEMB002C|+|2709' 'VT6436|1|MEMB002D|-|2501'
 'VT6436|1|MEMB002E|-|2699' 'VT6436|1|MEMB002F|-|2670'
 'VT6436|1|MEMB003A|-|2262' 'VT6436|1|MEMB003B|+|2585'
 'VT6436|1|MEMB003C|-|2741' 'VT6436|1|MEMB003D|-|2312'
 'VT6436|1|MEMB003F|-|2611' 'VT6436|1|MEMB004A|+|2422'
 'VT6436|1|MEMB004B|+|2551' 'VT6436|1|MEMB004E|-|2496'
 'VT6436|1|MEMB005B|-|2784' 'VT6436|1|MEMB005D|+|2691'
 'VT6436|1|MEMB006A|+|2653' 'VT6436|1|MEMB006B|-|2624'
 'VT6436|1|MEMB006C|-|2759' 'VT6436|1|MEMB007B|-|2711'
 'VT6436|1|MEMB007C|-|2597' 'VT6436|1|MEMB007D|-|2947'
 'VT6436|1|MEMB008C|+|2558' 'VT6436|1|dkik|+|2321']


,species,Unnamed: 0,score,raw_position,strand,align_position,motif
0,VT6436|1|MEMB003F|-|2611,53297,4.317153,2607,positive,3888,bcd_FlyReg.fm
1,VT6436|1|MEMB007B|-|2711,20397,4.317153,2707,positive,3888,bcd_FlyReg.fm
2,VT6436|1|MEMB007B|-|2711,20397,4.317153,2707,positive,3888,bcd_FlyReg.fm
3,VT6436|1|MEMB003A|-|2262,93881,4.317153,2258,positive,3888,bcd_FlyReg.fm
4,VT6436|1|MEMB004B|+|2551,74051,4.317153,2547,positive,3888,bcd_FlyReg.fm
5,VT6436|1|MEMB004E|-|2496,58283,4.317153,2492,positive,3888,bcd_FlyReg.fm
6,VT6436|1|MEMB002B|-|2408,10143,4.317153,2404,positive,3888,bcd_FlyReg.fm
7,VT6436|1|MEMB004A|+|2422,14981,4.317153,2418,positive,3888,bcd_FlyReg.fm
8,VT6436|1|MEMB002E|-|2699,124481,4.317153,2695,positive,3888,bcd_FlyReg.fm
9,VT6436|1|MEMB006B|-|2624,103741,4.317153,2620,positive,3888,bcd_FlyReg.fm


In [43]:
def randomize_position(max_align_positions):
    rand_pos = []
    for i in np.arange(len(max_align_positions)):
        random_index = random.randint(0, max_align_positions[i])
        rand_pos += [random_index]
    return rand_pos

In [53]:
def randomize_file(seq_length, species_count, no_thresh_path):
    """ 
    seq_length: The length of the sequence that we want
    species_count: The number of randomized species we want in our outputted csv
    no_thresh_path: The path to the non thresholded file
    path_to_raw: The path of the raw file for the region we want

    Returns a csv with randomly chosen species, alignment positions, strands, and the corresponding 
    enumerated species.
    """
    #get non thresholded file
    no_thresh = pd.read_csv(no_thresh_path) 
    species_array = no_thresh.drop_duplicates("species")["species"].values
    #randomly generate the species
    randomized_species = randomize_species(species_count, species_array)
    #group by species.max() and merge with randomly picked species
    subsetted = no_thresh.groupby("species").max()
    rand_spec = pd.DataFrame(randomized_species, columns = ["species"])
    subsetted = pd.merge(rand_spec, subsetted, on = ['species'])
    max_align_positions = subsetted["align_position"].values
    #randomly generate index positions
    rand_pos = randomize_position(max_align_positions)
    #drop uneeded columns
    subsetted = subsetted.drop(columns = ['score', 'motif', 'raw_position', 'Unnamed: 0'])
    subsetted['align_position'] = np.array(rand_pos)
    subsetted['strand'] = np.array(randomize_strand(len(rand_pos)))
    no_thresh = no_thresh.drop(columns = ['motif', 'Unnamed: 0', 'score']) 
    #merge with non thresholded file to enumerate all species
    result = pd.merge(subsetted, no_thresh, on = ['align_position', 'strand'])
    return result

In [54]:
randomize_file(5, 15, "~/motif_extraction/data/sub_no_map_motif_bcd_no_threshold/VT6436.fa.csv")

,species_x,strand,align_position,species_y,raw_position
0,VT6436|1|MEMB006C|-|2759,positive,915,VT6436|1|MEMB002A|+|2701,739
1,VT6436|1|MEMB006C|-|2759,positive,915,VT6436|1|MEMB002B|-|2408,719
2,VT6436|1|MEMB006C|-|2759,positive,915,VT6436|1|MEMB002C|+|2709,737
3,VT6436|1|MEMB006C|-|2759,positive,915,VT6436|1|MEMB002D|-|2501,751
4,VT6436|1|MEMB006C|-|2759,positive,915,VT6436|1|MEMB002E|-|2699,721
5,VT6436|1|MEMB006C|-|2759,positive,915,VT6436|1|MEMB002F|-|2670,753
6,VT6436|1|MEMB006C|-|2759,positive,915,VT6436|1|MEMB003A|-|2262,718
7,VT6436|1|MEMB006C|-|2759,positive,915,VT6436|1|MEMB003B|+|2585,759
8,VT6436|1|MEMB006C|-|2759,positive,915,VT6436|1|MEMB003C|-|2741,760
9,VT6436|1|MEMB006C|-|2759,positive,915,VT6436|1|MEMB003D|-|2312,698


In [55]:
def raw_string(region, seq_length, species_count, no_thresh_path, path_to_raw, output_path):
    """ 
    region: The region number ex: VT11048
    seq_length: The length of the sequence that we want
    species_count: The number of randomized species we want in our outputted csv
    no_thresh_path: The path to the non thresholded file
    path_to_raw: The path of the raw file for the region we want
    output_path: The path where you want the csv saved

    Returns a csv saved under output_path/*region*
    """
    
    result = randomize_file(seq_length, species_count, no_thresh_path, path_to_raw)
    print(result)
    num_reg = region.split('.')[0]
    if num_reg[0] == "_":
        num_reg = num_reg[1:]
    reg_path = path_to_raw + "/outlier_rm_with_length_"+str(num_reg)+".fa"
    record_dict = SeqIO.to_dict(SeqIO.parse(reg_path, "fasta"))
    
    sequences = []
    before = []
    after = []
    
    for index, row in result.iterrows():
        speci = row['species_y']
        pos = row['raw_position']
        strand = row['strand']
        seq = record_dict[speci]
        if strand == 'negative':
            sequences.append(getNegative(seqToString(seq[pos:pos + seq_length])))
            before.append(getNegative(seqToString(seq[pos - seq_length:pos])))
            after.append(getNegative(seqToString(seq[pos + seq_length:pos + seq_length + seq_length])))
        else:
            sequences.append(seqToString(seq[pos:pos + seq_length]))
            before.append(seqToString(seq[pos - seq_length:pos]))
            after.append(seqToString(seq[pos + seq_length:pos + seq_length + seq_length]))
    result['raw_seq'] = np.array(sequences)
    result['before_seq'] = np.array(before)
    result['after_seq'] = np.array(after)
    print(result)
    path = os.path.join(output_path, str(num_reg) + "_random.csv")
    print(path)
    result.to_csv(path)

In [56]:
raw_string("VT6436", 5, 15, "/Users/niharikadesaraju/motif_extraction/data/sub_no_map_motif_bcd_no_threshold/VT6436.fa.csv", "/Users/niharikadesaraju/motif_extraction/data/raw", "/Users/niharikadesaraju/motif_extraction/data/output/") 

TypeError: randomize_file() takes 3 positional arguments but 4 were given